### Notebook to run association analyses for HBN sample.
- Author: Dominik Kraft

- make sure to use correct conda environment or use __conda activate net_fusion__ before starting the jupyter notebook

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf


pd.options.mode.chained_assignment = None 

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

In [189]:
def partial_eta_square(model):
    """
    Function to calculate partial eta square from OLS model after fitting
    ---
    Input:
    - sm.model object after fitting the model
    ----
    Output:
    - no return value, but prints partial eta squares for each predictor in model
    """
    
    # transform model to anova table
    anova_table = sm.stats.anova_lm(model) 
    # index of anova table = predictor list
    predictors = anova_table.index.to_list()
    
    # exclude categorical predictors + residuals
    to_exclude = ("C(", "Resi")
    predictors = [p for p in predictors if not p.startswith(to_exclude)]
    
    #iterate over all predictors and calculate eta squared 
    for predictor in predictors:
        SS_predictor = anova_table.loc[predictor, 'sum_sq']
        SS_resid = anova_table.loc['Residual', 'sum_sq']
        partial_eta_squared = SS_predictor / (SS_predictor + SS_resid)
        print(f'Partial eta squared for {predictor}: {partial_eta_squared.round(5)}')

In [10]:
dic = np.load('../embedding_scripts/outputdict_hbn.npy',allow_pickle='TRUE').item()
# load output from main_hbn.py

In [153]:
## combine demos and predicted embedding
hbn = dic["demos"][1] 
hbn["prediction"] = np.squeeze(dic["predictions"]) # predicted embedding
hbn.rename(columns={"Identifiers":"subjectkey"},inplace=True)

## get puberty information

#### Note: we use data from LORIS + COINS releases to make sure to gather every available data 

- The first cells contain some data handling to align both datasets and eventually combine them 





In [121]:
### specify puberty columns for loris
pub_cols = ['PPS,PPS_F_01',
 'PPS,PPS_F_02',
 'PPS,PPS_F_03',
 'PPS,PPS_F_04','PPS,PPS_F_05',
 'PPS,PPS_F_06', "PPS,PPS_F_Score",
'PPS,PPS_M_01',
 'PPS,PPS_M_02',
 'PPS,PPS_M_03',
 'PPS,PPS_M_04',
 'PPS,PPS_M_05',
'PPS,PPS_M_06',
           "PPS,PPS_M_Score"]

In [132]:
loris_pub=pd.read_csv('../hbn/nonMRI/HBN_Petersen_neu1708.csv', sep = ',', skiprows=[1],
                usecols= pub_cols + ["Identifiers"])
                   # read header
    
loris_pub["Identifiers"] = ["sub-"+ x for x in loris_pub["Identifiers"]]
loris_pub["Identifiers"] =loris_pub["Identifiers"].str.replace(',assessment', '')
loris_pub.drop_duplicates(subset=["Identifiers"], inplace=True)

loris_pub.shape

(2842, 15)

In [133]:
### different column naming between loris and coins file 
col_map = [x.replace('PPS,', '') for x in loris_pub.columns if x.startswith("PPS")]
col_to_map = [x for x in loris_pub.columns if x.startswith("PPS")]
mapper_dic = dict(zip(col_to_map, col_map))
mapper_dic


## map loris column names to coin column names

{'PPS,PPS_F_01': 'PPS_F_01',
 'PPS,PPS_F_02': 'PPS_F_02',
 'PPS,PPS_F_03': 'PPS_F_03',
 'PPS,PPS_F_04': 'PPS_F_04',
 'PPS,PPS_F_05': 'PPS_F_05',
 'PPS,PPS_F_06': 'PPS_F_06',
 'PPS,PPS_F_Score': 'PPS_F_Score',
 'PPS,PPS_M_01': 'PPS_M_01',
 'PPS,PPS_M_02': 'PPS_M_02',
 'PPS,PPS_M_03': 'PPS_M_03',
 'PPS,PPS_M_04': 'PPS_M_04',
 'PPS,PPS_M_05': 'PPS_M_05',
 'PPS,PPS_M_06': 'PPS_M_06',
 'PPS,PPS_M_Score': 'PPS_M_Score'}

In [135]:
loris_pub.rename(mapper=mapper_dic, axis=1, inplace=True)
loris_pub.rename(columns={"Identifiers":"subjectkey"},inplace=True)
loris_pub[col_map] = loris_pub[col_map].replace('.', np.nan)
loris_pub[col_map] = loris_pub[col_map].astype("float64")


In [136]:
### COINS ###

coins_pub = pd.read_csv("../hbn/nonMRI/coins/assessment_data/9994_PPS_20220818.csv",
                      sep = ',', skiprows=[1])


coins_pub["subjectkey"] = ["sub-"+ x for x in coins_pub["EID"]] ### note: in loris file, variables are labeled differently

coins_pub["subjectkey"] =coins_pub["subjectkey"].str.replace(',assessment', '')

coins_pub.drop_duplicates(subset=["subjectkey"], inplace=True)

coins_pub = coins_pub[["subjectkey"]+col_map]
coins_pub.shape



(2450, 15)

In [137]:
pub_combined = pd.concat([coins_pub, loris_pub]) # concat both loris and coins
print(pub_combined.shape)
pub_combined.drop_duplicates(subset="subjectkey", inplace=True) # drop duplicates
print(pub_combined.shape)


(5292, 15)
(2843, 15)


In [155]:
pub_d = pd.merge(hbn, pub_combined, on="subjectkey") #merge embedding info and puberty info
pub_d.drop_duplicates(subset="subjectkey", inplace=True)
pub_d.shape

(1709, 30)

## Association Analyses




### get puberty information

#### puberty information in analogy to ABCD sample

In [156]:
fem = pub_d.loc[pub_d["Sex"]==1] ## subset for sex
men = pub_d.loc[pub_d["Sex"]==0]


## calculate gonadal puberty scores 
fem["gonadal"]= fem[["PPS_F_01", "PPS_F_04", "PPS_F_06"]].mean(axis=1, skipna=False)
men["gonadal"] = men[["PPS_M_01","PPS_M_04", "PPS_M_05"]].mean(axis=1, skipna=False)


## calculate adrenal puberty scores
fem["adrenal"]= fem[["PPS_F_02", "PPS_F_03"]].mean(axis=1, skipna=False)
men["adrenal"]= men[["PPS_M_02", "PPS_M_03"]].mean(axis=1, skipna=False)

## calculate average PDS scores
fem["PDS_mean"] =  fem[['PPS_F_01',
 'PPS_F_02',
 'PPS_F_03',
 'PPS_F_04',
 'PPS_F_06',]].mean(axis=1, skipna=False)
men["PDS_mean"] =  men[['PPS_M_01',
 'PPS_M_02',
 'PPS_M_03',
 'PPS_M_04',
 'PPS_M_05']].mean(axis=1, skipna=False)

## calculate PDS category score
fem["PDS_cat_score"] = fem[["PPS_F_02", "PPS_F_04"]].sum(axis=1, skipna=False) ##### error f2 statt f1 ### 
men["PDS_cat_score"] = men[["PPS_M_02","PPS_M_04", "PPS_M_05"]].sum(axis=1, skipna=False)

In [157]:
def female_categorical(row):
    '''
    female: body hair growth + breast development || using menarche info as follows:
    prepubertal = 2 + no menarche
    early pubertal = 3 + no menarche
    midpubertal =>3 + no menarche
    late pubertal <=7 + menarche
    postpubertal = 8 + menarche
    according to Herting et al (2021) Frontiers in Endocrinology
    '''
    if np.isnan(row["PDS_cat_score"])==False:
        
        if row["PPS_F_06"] == 0.0: #no
            if row["PDS_cat_score"] == 2.0:
                return "prepubertal"
            if row["PDS_cat_score"] == 3.0:
                return "early pubertal"
            if row["PDS_cat_score"] >= 3.0:
                return "midpubertal"
            
        elif row["PPS_F_06"] == 1.0: #yes
            if row["PDS_cat_score"] <= 7.0:
                return "late pubertal"
            if row["PDS_cat_score"] == 8.0:
                return "postpubertal"
                  
    elif np.isnan(row["PDS_cat_score"])==True:
        return np.nan
    
    
def male_categorical(row):
    '''
    male: body hair growth + facial hair + voice change
    prepubertal = 3 x
    early pubertal = 4 or 5 (no 3 point response)x
    midpubertal = 6-8 (no point 4 response) x
    late pubertal = 9-11 
    postpubertal = 12 (all 4 point)
    according to Herting et al (2021) Frontiers in Endocrinology
    with minor adjustment to not create cases for which category is not
    well defined (see paper)
    '''
    if np.isnan(row["PDS_cat_score"])==False:
        
        if row["PDS_cat_score"] == 3.0:
            return "prepubertal"
        
        if 4.0 <= row["PDS_cat_score"] <= 5.0:
            return "early pubertal"
            
        if 6.0 <= row["PDS_cat_score"] <= 8.0:
            return "midpubertal"
            
        if 9.0 <= row["PDS_cat_score"] <= 11.0:
                return "late pubertal"
            
        if row["PDS_cat_score"] == 12.0:
            return "postpubertal"
        
    elif np.isnan(row["PDS_cat_score"])==True:
        return np.nan

In [158]:
fem["PDS_category"] = fem.apply(lambda row: female_categorical(row), axis=1)
men["PDS_category"] = men.apply(lambda row: male_categorical(row), axis=1)

pub_all = pd.concat([fem, men])
pub_all.shape

(1709, 35)

In [165]:
tmp1 = pub_all.copy() # copy dataframe 

### Psychopathology Model


In [194]:
psycho_models = [
    'prediction ~ summed + Age + C(Scan_Location)',
    'prediction ~ PDS_mean + summed + Age + C(Scan_Location)',
    'prediction ~ PDS_mean + summed + summed:PDS_mean + Age + C(Scan_Location)']

for s in [0,1]:
    if s == 0:
        print("MALE")
    elif s == 1:
        print("FEMALE")
    
    for m in psycho_models:
        
        
        model = smf.ols(formula=m, data=tmp1.loc[tmp1["Sex"]==s]).fit()
        summary = model.summary()

        print(m)
        print(int(model.nobs))
        print(summary.tables[1])
        print(model.params[-2], model.pvalues[-2])
        print(model.params[-1], model.pvalues[-1])
        print("\n")
        partial_eta_square(model)
        print("----------------")
        print("\n")
        


MALE
prediction ~ summed + Age + C(Scan_Location)
1113
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                 0.2236      0.196      1.143      0.253      -0.160       0.607
C(Scan_Location)[T.2]     0.0544      0.164      0.332      0.740      -0.267       0.376
C(Scan_Location)[T.3]     0.0757      0.164      0.462      0.644      -0.246       0.398
C(Scan_Location)[T.4]     0.0429      0.204      0.211      0.833      -0.356       0.442
summed                    0.0393      0.022      1.773      0.077      -0.004       0.083
Age                      -0.0514      0.011     -4.843      0.000      -0.072      -0.031
0.039315089323482086 0.07655844586148765
-0.051418941340083686 1.4572384100851914e-06


Partial eta squared for summed: 0.00123
Partial eta squared for Age: 0.02075
----------------


prediction ~ PDS_mean + summed + Age + C(